In [1]:
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phrases, Phraser

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import glob
import os
from os import path
import string
import numpy
import re
from __future__ import absolute_import, division, print_function
import pandas as pd
import multiprocessing
import gensim
import random
from tqdm import tqdm
from sklearn.manifold import TSNE
from gensim.models import KeyedVectors
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [86]:
os.chdir("D://Scriptie//Data//lines//cleaned")
all_txt = glob.glob('*.txt')

In [79]:
def file_len(fname):
    with open(fname, encoding = 'utf-8') as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def ExtractSampleSentences(start_year, period, sample_size):
    list_txt_range = [t for t in all_txt if int(t[0:4]) >= start_year and int(t[0:4]) < start_year + period]
    
    # Get Size of Range
    range_size = 0
    numb_lines = 0
    
    for txt in list_txt_range:
        statinfo = os.stat(txt)
        range_size += int(statinfo.st_size)
        numb_lines += file_len(txt)
    
    range_size = round(range_size/ 1000000)
        
    sample_size = sample_size + 1
    
    if range_size > sample_size:
        tot_lines_samp = round(sample_size * numb_lines / range_size)
        print('an average of ' + str(tot_lines_samp) + ' lines a file is going to be imported in period ' + str(start_year))
        
        lines_period = list()
        
        for txt in list_txt_range:
            with open(txt, encoding = 'utf-8') as f:
                content = f.read().splitlines()
                
                if len(content) <= round(tot_lines_samp / 10):
                    for line in content:
                        lines_period.append(line.split())
                
                if len(content) > round(tot_lines_samp / 10):
                    content = random.sample(content, round(tot_lines_samp / 10))
                    for line in content:
                        lines_period.append(line.split())
    
    else:
        print('period is small enough')
        lines_period = list()
        
        for txt in list_txt_range:
            with open(txt, encoding = 'utf-8') as f:
                content = f.readlines()
                for line in content:
                        lines_period.append(line.split())
    
    lines_period = [line for line in lines_period if len(line) > 1]
    return lines_period


In [108]:
def Txt_Lines(txt):
    with open(txt, encoding = 'utf-8') as f:
            content = f.read().split('\n')
    
    lines = list()
    
    for line in content:
        tmp = line.split(' ')
        lines.append(tmp)
    
    return lines

def build_phrases(sentences):
    phrases = Phrases(sentences,
                      min_count=3,
                      threshold=5,
                      progress_per=1000)
    return Phraser(phrases)

def sentence_to_bi_grams(phrases_model, sentence):
    return ' '.join(phrases_model[sentence])

def txt_to_bigrams(input_fn, output_fn):
    
    sentences = Txt_Lines(input_fn)
    phrases_model = build_phrases(sentences)
    
    with open(output_fn, 'w', encoding = 'utf-8') as f:
            for item in sentences:
                bgs = sentence_to_bi_grams(phrases_model, item)
                f.write("%s\n" % bgs)

In [110]:
for period in range(1815,1914,10):
    sentences = ExtractSampleSentences(period,10,350)
    phrases_model = build_phrases(sentences)
    
    output_fn = str(period) + "_10yp_bigrams.txt"
    
    with open(output_fn, 'w', encoding = 'utf-8') as f:
            for item in sentences:
                bgs = sentence_to_bi_grams(phrases_model, item)
                f.write("%s\n" % bgs)

period is small enough
an average of 213895 lines a file is going to be imported in period 1825
an average of 203986 lines a file is going to be imported in period 1835
an average of 249262 lines a file is going to be imported in period 1845
an average of 269267 lines a file is going to be imported in period 1855
an average of 241412 lines a file is going to be imported in period 1865
an average of 257565 lines a file is going to be imported in period 1875
an average of 249031 lines a file is going to be imported in period 1885
an average of 249769 lines a file is going to be imported in period 1895
an average of 295495 lines a file is going to be imported in period 1905


In [112]:
all_txt = glob.glob('*bigrams.txt')
all_txt

['1815_10yp_bigrams.txt',
 '1825_10yp_bigrams.txt',
 '1835_10yp_bigrams.txt',
 '1845_10yp_bigrams.txt',
 '1855_10yp_bigrams.txt',
 '1865_10yp_bigrams.txt',
 '1875_10yp_bigrams.txt',
 '1885_10yp_bigrams.txt',
 '1895_10yp_bigrams.txt',
 '1905_10yp_bigrams.txt']

In [115]:
count = 0

for txt in all_txt:
    if count == 0:
        count += 1
        sentences = Txt_Lines(txt)
        model = gensim.models.Word2Vec(sentences, min_count=10, workers=6)
        os.chdir("D://Scriptie//Data//models")
        model.save(str(txt[:-4])+"_model.w2v")
        print("Model saved, on to the next\n")
        os.chdir("D://Scriptie//Data//lines//cleaned")

    if count > 0:
        sentences = Txt_Lines(txt)
        model.build_vocab(sentences, update=True)
        model.train(sentences, total_examples = model.corpus_count, start_alpha = model.alpha, end_alpha = model.min_alpha, epochs = model.iter)
        os.chdir("D://Scriptie//Data//models")
        model.save(str(txt[:-4])+"_model.w2v")
        print("Model saved, on to the next\n")
        os.chdir("D://Scriptie//Data//lines//cleaned")

Model saved, on to the next



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  app.launch_new_instance()


Model saved, on to the next

Model saved, on to the next

Model saved, on to the next

Model saved, on to the next

Model saved, on to the next

Model saved, on to the next

Model saved, on to the next

Model saved, on to the next

Model saved, on to the next

Model saved, on to the next



In [120]:
model.most_similar("ons_werelddeel", topn=25)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('westelijk_europa', 0.8218541145324707),
 ('dit_werelddeel', 0.805173397064209),
 ('geheel_europa', 0.759597659111023),
 ('westeuropa', 0.7475301027297974),
 ('enropa', 0.7467577457427979),
 ('middeneuropa', 0.7319943904876709),
 ('het_oosten', 0.7141872048377991),
 ('ons_vaderland', 0.699781060218811),
 ('europas', 0.6976679563522339),
 ('luropa', 0.6971043348312378),
 ('furopa', 0.6947068572044373),
 ('zuideuropa', 0.6926820874214172),
 ('oostelijk_europa', 0.6906366348266602),
 ('europa', 0.679642915725708),
 ('kuropa', 0.6784006357192993),
 ('het_westen', 0.6780215501785278),
 ('vele_landen', 0.6766130328178406),
 ('den_aardbodem', 0.6750328540802002),
 ('bet_oosten', 0.67425537109375),
 ('gansch_europa', 0.6705217361450195),
 ('naburige_rijken', 0.6697952151298523),
 ('dit_land', 0.6697531938552856),
 ('naburige_landen', 0.6677975654602051),
 ('deze_eeuw', 0.6597380638122559),
 ('werelddeel', 0.6580584049224854)]

In [122]:
os.chdir("D://Scriptie//Data//models")
m = model.wv.load("1835_10yp_bigrams_model.w2v")

In [129]:
m.most_similar("europa",topn=20 )

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('enropa', 0.8588883280754089),
 ('der_wereld', 0.8049423694610596),
 ('ons_werelddeel', 0.796471893787384),
 ('dit_werelddeel', 0.7676461338996887),
 ('geheel_europa', 0.76515793800354),
 ('dit_land', 0.760669469833374),
 ('azië', 0.7446967363357544),
 ('het_schiereiland', 0.7309390306472778),
 ('frankrijk', 0.7254589796066284),
 ('volken', 0.7207484245300293),
 ('gansch_europa', 0.7193469405174255),
 ('het_oosten', 0.7112278342247009),
 ('engeland', 0.704196572303772),
 ('alle_volken', 0.6990007162094116),
 ('ons_land', 0.6968019604682922),
 ('nederland', 0.6903970837593079),
 ('volkeren', 0.688761830329895),
 ('andere_staten', 0.6878013610839844),
 ('natiën', 0.6872297525405884),
 ('alle_volkeren', 0.6845576763153076)]